# Capstone Project

In [ ]:
import pandas as pd
import numpy as np

employee_data = [
    ['A001', 'John Alter',       'M', 'Paris',     25],
    ['A002', 'Alice Luxumberg',  'F', 'London',    27],
    ['A003', 'Tom Sabestine',    'M', 'Berlin',    29],
    ['A004', 'Nina Adgra',       'F', 'Newyork',   31],
    ['A005', 'Amy Johny',        'F', 'Madrid',    30],
]

# Define column names
columns = ['ID', 'Name', 'Gender', 'City', 'Age']

# Create DataFrame
df_employee = pd.DataFrame(employee_data, columns=columns)

# Show the DataFrame
print(df_employee)

project_data = [
    ['A001', 'Project 1', 1002000, 'Finished'],
    ['A002', 'Project 2', 2000000, 'Ongoing'],
    ['A003', 'Project 3', 4500000, 'Finished'],
    ['A004', 'Project 4', 5500000, 'Ongoing'],
    ['A005', 'Project 5', np.nan,    'Finished'],
    ['A002', 'Project 6', 680000, 'Failed'],
    ['A005', 'Project 7', 400000, 'Finished'],
    ['A003', 'Project 8', 350000, 'Failed'],
    ['A001', 'Project 9', np.nan,    'Ongoing'],
    ['A003', 'Project 10', 300000, 'Finished'],
    ['A001', 'Project 11', 2000000, 'Failed'],
    ['A004', 'Project 12', 1000000, 'Ongoing'],
    ['A004', 'Project 13', 3000000, 'Finished'],
    ['A005', 'Project 14', 200000, 'Finished']
]

# Define the column names
columns = ['ID', 'Project', 'Cost', 'Status']

# Create the DataFrame
df_project = pd.DataFrame(project_data, columns=columns)

# Show the DataFrame
print(df_project)

seniority_data = [
    ['A001', 2],
    ['A002', 2],
    ['A003', 3],
    ['A004', 2],
    ['A005', 3]
]

# Define columns
columns = ['ID', 'Designation Level']

# Create DataFrame
df_seniority = pd.DataFrame(seniority_data, columns=columns)

# Display it
print(df_seniority)

# Save to CSV
df_employee.to_csv('Employee.csv', index=False)
df_project.to_csv('Project.csv', index=False)
df_seniority.to_csv('Seniority.csv', index=False)

ModuleNotFoundError: No module named 'pandas'

## Task 2-10

In [ ]:
# -------------------------
# Task 2: Fill missing cost values using running average
# -------------------------
# Read the Project.csv file into a pandas DataFrame
df_project = pd.read_csv("Project.csv")

# Convert the 'Cost' column to a Python list for easier manipulation
costs = df_project['Cost'].tolist()

# Iterate through each element in the costs list using index
for i in range(len(costs)):
    # Check if the current cost value is NaN (missing)
    if pd.isna(costs[i]):
        # Initialize counters for calculating running average
        count = 0  # Count of non-missing values found so far
        total = 0  # Sum of non-missing values found so far
        
        # Look at all previous values (from index 0 to current index i-1)
        for j in range(i):
            # Check if the value at index j is not missing
            if not pd.isna(costs[j]):
                # Add the value to running total
                total += costs[j]
                # Increment the count of valid values
                count += 1
        
        # Calculate running average: if count > 0, use average; otherwise use 0
        costs[i] = total / count if count != 0 else 0

# Replace the original 'Cost' column with the updated list (missing values filled)
df_project['Cost'] = costs

# -------------------------
# Task 3: Split name into First Name and Last Name
# -------------------------
# Read the Employee.csv file into a pandas DataFrame
df_employee = pd.read_csv("Employee.csv")

# Split the 'Name' column on space character, create two new columns
# n=1 means split only on first space, expand=True creates separate columns
df_employee[['First Name', 'Last Name']] = df_employee['Name'].str.split(' ', n=1, expand=True)

# Remove the original 'Name' column as it's no longer needed
df_employee.drop(columns=['Name'], inplace=True)

# -------------------------
# Task 4: Join all three DataFrames into "Final"
# -------------------------
# Read the Seniority.csv file into a pandas DataFrame
df_seniority = pd.read_csv("Seniority.csv")

# Perform inner joins: first merge employee with seniority data on 'ID' column
# Then merge the result with project data on 'ID' column
final_df = df_employee.merge(df_seniority, on='ID').merge(df_project, on='ID')

# Rename 'Designation Level' column to 'Designation' for clarity
final_df.rename(columns={'Designation Level': 'Designation'}, inplace=True)

# -------------------------
# Task 5: Add 5% Bonus for finished projects
# -------------------------
# Create a new 'Bonus' column by applying a lambda function to each row
# If project status is 'finished' (case-insensitive), bonus = 5% of cost; otherwise 0
final_df['Bonus'] = final_df.apply(
    lambda row: 0.05 * row['Cost'] if row['Status'].lower() == 'finished' else 0, axis=1
)

# -------------------------
# Task 6: Demote designation level by 1 for failed projects and remove level > 4
# -------------------------
# Use boolean indexing to find rows where Status is 'failed' (case-insensitive)
# Subtract 1 from the 'Designation' column for these rows
final_df.loc[final_df['Status'].str.lower() == 'failed', 'Designation'] -= 1

# Filter out rows where Designation level is greater than 4
final_df = final_df[final_df['Designation'] <= 4]

# -------------------------
# Task 7: Add Mr./Mrs. prefix and drop Gender
# -------------------------
# Define a function to add appropriate title based on gender
def add_title(row):
    # Return 'Mr.' + First Name if Gender is 'M', otherwise 'Mrs.' + First Name
    return ('Mr. ' if row['Gender'] == 'M' else 'Mrs. ') + row['First Name']

# Apply the add_title function to each row and update the 'First Name' column
final_df['First Name'] = final_df.apply(add_title, axis=1)

# Remove the 'Gender' column as it's no longer needed after adding titles
final_df.drop(columns=['Gender'], inplace=True)

# -------------------------
# Task 8: Promote designation by 1 if age > 29
# -------------------------
# Apply a lambda function to each row to conditionally promote employees
# If age > 29, add 1 to current designation; otherwise keep current designation
final_df['Designation'] = final_df.apply(
    lambda row: row['Designation'] + 1 if row['Age'] > 29 else row['Designation'],
    axis=1
)

# -------------------------
# Task 9: Total project cost per employee
# -------------------------
# Group by employee ID and First Name, then sum the Cost column for each group
total_proj_cost = final_df.groupby(['ID', 'First Name'])['Cost'].sum().reset_index()

# Rename the summed 'Cost' column to 'Total Cost' for clarity
total_proj_cost.rename(columns={'Cost': 'Total Cost'}, inplace=True)

# Print header for the total project cost results
print("\n--- Total Project Cost Per Employee ---")
# Display the total project cost per employee
print(total_proj_cost)

# -------------------------
# Task 10: Print employees whose city name contains the letter "o"
# -------------------------
# Filter rows where 'City' column contains the letter 'o' (case-insensitive)
employees_with_o = final_df[final_df['City'].str.contains('o', case=False)]

# Print header for employees with 'o' in city name
print("\n--- Employees whose City contains 'o' ---")
# Display the filtered employees
print(employees_with_o)